In [54]:
!pip install langchain_core
!pip install langchain
!pip install langchain_community
!pip install arxiv
!pip install pypdf
!pip install faiss-gpu
!pip install groq
!pip install fastembed
!pip install wikipedia
!pip install python-dotenv
!pip install langchain-groq
!pip install sentence-transformers
!pip install -U langchain-huggingface

In [55]:
!pip install pypdf

In [56]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [57]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [58]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from fastembed import TextEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import DirectoryLoader

In [59]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
api = os.getenv("GROQ_API_KEY")
Model = "llama-3.1-70b-versatile"
llm = ChatGroq(api_key=api,model=Model)

In [60]:
def load_pdf(data):
  loader = DirectoryLoader(data,glob="*.pdf",loader_cls = PyPDFLoader)
  documents = loader.load()
  return documents

extracted_data = load_pdf("data/")

In [61]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [62]:
text_chunks = text_split(extracted_data)
print(f"Length of the Chunk {len(text_chunks)}")

Length of the Chunk 4445


In [63]:
def embedding_model():
  model_name = "sentence-transformers/all-MiniLM-L6-v2"
  embedding = HuggingFaceEmbeddings(model_name = model_name)
  return embedding

In [64]:
embeddings = embedding_model()

In [65]:
sample_embedding = embeddings.embed_query("CDK")
print(f"Length of the sample_embedding{len(sample_embedding)}")
print(f"Example of how the embedding model looks like : {sample_embedding}")

Length of the sample_embedding384
Example of how the embedding model looks like : [-0.07624874264001846, 0.11039117723703384, -0.08735432475805283, -0.037289027124643326, -0.03572263568639755, -0.059426192194223404, 0.09830831736326218, 0.07049110531806946, 0.014280569739639759, 0.03800587356090546, 0.0020426714327186346, -0.10246293991804123, 0.015605214051902294, -0.031947873532772064, -0.012786182574927807, 0.042728241533041, -0.01973150297999382, 0.008957058191299438, -0.06602046638727188, -0.031694408506155014, -0.11810354143381119, -0.04171375185251236, -0.024348357692360878, -0.015065202489495277, -0.02605675905942917, -0.016803612932562828, 0.03977925330400467, 0.07697451859712601, -0.05818972736597061, -0.0630159005522728, -0.000793376297224313, 0.03802512586116791, 0.02524895779788494, 0.002769293962046504, 0.026908647269010544, 0.056114643812179565, -0.04573231190443039, -0.014879941940307617, -0.02331913262605667, -0.039954543113708496, 0.01325595285743475, -0.0584378279745

In [66]:
def vector_embedding():
  store = FAISS.from_documents(text_chunks,embeddings)
  embedding = store.as_retriever()
  return embedding
vector_database = vector_embedding()

In [67]:
prompt_template = """
Hello mister You are the AI, your name is BrainWave okayy then if user asks the question you need to give the related answer only clearly check the answer whether it is correct or not
if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner
context:{context}
question:{question}
"""

In [68]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(template = prompt_template , input_variables=["context","query"],chain_type_kwargs={"prompt":prompt_template})

In [69]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm = llm,
            chain_type="stuff",
            retriever=vector_database,
            chain_type_kwargs={"prompt":prompt})

In [132]:
user_input = input("Input Prompt: ")
result = qa_chain({"query": user_input})
final_result = result["result"]
print(final_result)

Input Prompt: How can i overcome malaria
To overcome malaria, the following steps can be taken:

1. **Diagnosis**: If you suspect you have malaria, get tested using rapid diagnostic tests (dipsticks) or microscopy. If these tests are not available, diagnosis can be made based on clinical symptoms.

2. **Medicine Treatment**: Take antimalarial drugs as prescribed by a healthcare professional. For non-severe malaria, oral treatment is usually given. However, if you vomit within the first hour of taking the medication, repeat the treatment.

3. **General Measures**: Provide supportive and symptomatic relief, monitor for complications, and ensure adequate hydration.

4. **Rest and Recovery**: Rest and recover in a comfortable environment, avoiding excessive physical activity.

5. **Prevention**: To prevent malaria, take antimalarial drugs before, during, and after traveling to areas where malaria is common. Use mosquito control measures such as insecticide-treated bed nets, and wear protec

In [80]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(vector_database, "qa_tool","Hello mister You are the AI, your name is BrainWave okayy then if user asks the question you need to give the related answer only clearly check the answer whether it is correct or not if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner")

In [81]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_api = ArxivAPIWrapper()
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)

In [82]:
tools = [arxiv,retriever_tool]
tools

[ArxivQueryRun(),
 Tool(name='qa_tool', description="Hello mister You are the AI, your name is BrainWave okayy then if user asks the question you need to give the related answer only clearly check the answer whether it is correct or not if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7b0610648a60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b05f81ec760>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7b0610648b80>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_communi

In [98]:
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder

prompt_template = """
You are a helpful assistant. Your work is to act as a doctor here. If a user asks you any question regarding pain or health issues,
you need to provide them with medicine and assist them with instructions on how to take it (e.g., morning 1, evening 1, night 1).
You can retrieve information from the PDF, research papers, or even from the web.

Don't provide anything further from this. If they ask you about other questions than health, medicine, cure, disease,
and the name of medicine, you need to answer BrainWave has no idea other than Health Related Questions.

Hello mister, you are the AI, your name is BrainWave. If a user asks the question, you need to give the related answer only.
Clearly check the answer whether it is correct or not. If it's not correct, just inform BrainWave AI doesn't know about this
and don't make up an answer. Say "I don't know about this" in a beautiful manner.Don't provide where you found the answer and finish the answer in 1 small para including all and don't provide the page name and book name and all just the answer is enough
and remember you need to finish your answer within 100 words

context: {context}
question: {query}
{agent_scratchpad}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "query"],
    partial_variables={"agent_scratchpad": MessagesPlaceholder(variable_name="agent_scratchpad")} # Define agent_scratchpad as a MessagesPlaceholder
)

In [99]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [100]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['context', 'query'], partial_variables={'agent_scratchpad': MessagesPlaceholder(variable_name='agent_scratchpad')}, template='\nYou are a helpful assistant. Your work is to act as a doctor here. If a user asks you any question regarding pain or health issues, \nyou need to provide them with medicine and assist them with instructions on how to take it (e.g., morning 1, evening 1, night 1). \nYou can retrieve information from the PDF, research papers, or even from the web.\n\nDon\'t provide anything further from this. If they ask you about other questions than health, medicine, cure, disease, \nand the name of medicine, you need to answer BrainWave has no idea other than Health Related Questions.\n\nHello mister, you are the AI, your name is BrainWave. If a use

In [116]:
response = agent_executor.invoke({
    "query": "How can i overcome malaria",
    "context": "" })



> Entering new AgentExecutor chain...

Invoking: `qa_tool` with `{'query': 'Malaria treatment'}`


dipsticks, these can be used to diagnose malaria within 10–15 minutes. 
 
Note : 
If neither microscopy nor rapid tests are available diagnosis should be made on the basis of clinical symptoms. 
A blood smear should be made and sent for microscopic examination. 
One negative malaria test does not exclude the diagnosis of malaria.   
GENERAL MEASURES 
Provide supportive and symptomatic relief. 
Monitor for complications. 
Ensure adequate hydration. 
All patients with P. falciparum  malaria should be carefully observed for the first 
24 hours. 
 
MEDICINE TREATMENT 
 
 
Vomiting is common in patients with malaria. Give all first doses of drugs 
under supervision and observe patients for at least an hour. Repeat the 
treatment if the patient vomits within the first hour. 
Vomiting oral treatment is one of the commonest reasons for treatment 
failure.

In recent years, some strains of Plasm